<a href="https://www.kaggle.com/code/tejzdev/synapse-scholar?scriptVersionId=280073899" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Setup and API Configuration

This is the initialization cell. It handles all necessary groundwork before any LLM calls can be made. First, it imports essential Python libraries like os, asyncio, time, and datetime. Crucially, it imports google.generativeai to interface with the Gemini API. Since this notebook is running in a secure environment, it uses **kaggle_secrets** to securely retrieve the **GOOGLE_API_KEY**. The script then sets this key as an environment variable and configures the genai client, making the API ready to use. Finally, **nest_asyncio.apply()** is called to patch the environment, which is often required to run asynchronous code (like API calls) smoothly within a Jupyter notebook.

In [1]:
# This code is written entirely by Tejas K

import os
import google.generativeai as genai
import nest_asyncio
import asyncio
import time
from datetime import datetime
import json
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
nest_asyncio.apply()

# Load API key from Kaggle Secrets
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

print("Google Gemini configured successfully!")

Google Gemini configured successfully!


## LLM Wrapper Function

This cell defines the primary, reusable interface for interacting with the Gemini model. It sets a constant **MODEL_NAME** to **"gemini-2.0-flash"**. This model is chosen for its balance of high quality and low latency, making it ideal for the rapid interactions required in a multi-agent system. The asynchronous function **llm(prompt)** acts as a simple wrapper: it initializes the specified Gemini model, sends the user's prompt using **generate_content()**, and returns the plain generated text, streamlining the process for all subsequent agent calls.

In [2]:
MODEL_NAME = "gemini-2.0-flash"

async def llm(prompt):
    """Simple wrapper for Gemini calls."""
    response = genai.GenerativeModel(MODEL_NAME).generate_content(prompt)
    return response.text

## Context
This cell establishes variables and helper functions crucial for maintaining the conversation's state and history. The **memory_bank** is a simple list used for general logging or storing high-level interaction history. The session_state dictionary is the central hub for the user's session, holding parameters like the difficulty level for quizzes and a list of messages for context. The **compact_context** function is a crucial piece of Context Engineering. It ensures that only the last eight messages (max_len=8) are kept in the context history. This mechanism prevents the LLM from processing an excessively long chat history, which saves on token usage and dramatically reduces the response latency.

In [3]:
memory_bank = []
session_state = {
    "messages": [],
    "difficulty": "medium",
    "user_name": None
}

def add_to_memory(entry):
    memory_bank.append({
        "timestamp": datetime.now().isoformat(),
        "data": entry
    })

def compact_context(messages, max_len=8):
    """Context engineering: keep last N messages."""
    return messages[-max_len:]

## Logs and Metrics
To monitor and debug the complex agent flow, this cell sets up two tracking systems. The logs list stores detailed, time-stamped records of every major event, especially when an agent is triggered. The metrics dictionary tracks key statistics like **total_agent_calls** and **tool_calls**, providing insight into the system's performance and component usage. The log function provides a convenient way to add entries to the logs list and print a real-time console message.

In [4]:
logs = []
metrics = {
    "total_agent_calls": 0,
    "tool_calls": 0,
    "parallel_runs": 0,
    "sequential_runs": 0
}

def log(event, extra=None):
    logs.append({
        "event": event,
        "extra": extra,
        "time": datetime.now().isoformat()
    })
    print(f"[LOG] {event}")

## Some external tools
**Google Search(query):** This function uses an external, public search API (aiohttp is used for the web request) to fetch up-to-date information from the internet. It increments the tool_calls metric.

**calculator(expression):** This function uses Python's built-in eval() to execute mathematical expressions directly, providing highly reliable results for math queries. It also increments the tool_calls metric and includes basic error handling for invalid input.

In [5]:
import aiohttp

async def google_search(query):
    """Free search using a public search API."""
    metrics["tool_calls"] += 1
    url = f"https://ddg-api.herokuapp.com/search?query={query}"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            data = await resp.json()
            return data.get("results", [])

async def calculator(expression):
    metrics["tool_calls"] += 1
    try:
        return eval(expression)
    except:
        return "Invalid expression"

## Defining Agents
**agent_query_understanding(user_input):** This is the Intent Router Agent. Its sole purpose is to classify the user's input into one of six categories (explain, summarize, quiz, research, chat, math). The outcome of this agent call dictates which path the program follows.

**agent_explainer(question):** Provides simple, direct explanations of concepts.

**agent_summary(text):** Produces concise summaries suitable for revision.

**agent_quiz(topic, difficulty):** Generates a 5-question quiz based on the user's topic and the current session difficulty setting.

**agent_research(query):** This is the Grounding Agent. It first calls the external Google Search tool (Cell 5) to gather search results, then sends those results to the LLM to synthesize a coherent, grounded answer.

**agent_evaluator(answer):** A defined agent for the purpose of assessing student answers, although it is not currently connected to the main router flow.

In [6]:
async def agent_query_understanding(user_input):
    metrics["total_agent_calls"] += 1
    log("Query Understanding Agent triggered")

    prompt = f"""
    You are the Query Understanding Agent for Synapse Scholar.
    Classify the user message into one category:
    (explain, summarize, quiz, research, chat, math)

    Message: {user_input}

    Return ONLY the category.
    """
    return await llm(prompt)

async def agent_explainer(question):
    metrics["total_agent_calls"] += 1
    log("Explainer Agent triggered")
    return await llm(f"Explain this concept simply for a student:\n{question}")

async def agent_summary(text):
    metrics["total_agent_calls"] += 1
    log("Summarizer Agent triggered")
    return await llm(f"Summarize this text for revision:\n{text}")

async def agent_quiz(topic, difficulty):
    metrics["total_agent_calls"] += 1
    log("Quiz Generator Agent triggered")
    return await llm(f"Create a {difficulty} level quiz (5 questions) on:\n{topic}")

async def agent_research(query):
    metrics["total_agent_calls"] += 1
    log("Research Agent triggered")

    results = await google_search(query)
    text = "\n".join([r.get("title","") + ": " + r.get("body","") for r in results])
    return await llm(f"Based on this data, provide an answer:\n{text}")

async def agent_evaluator(answer):
    metrics["total_agent_calls"] += 1
    log("Evaluator Agent triggered")
    return await llm(f"Evaluate the student answer:\n{answer}")

## The Router

The **router(user_input)** function is the system's Control Plane.

It updates the **session_state** with the new input and compacts the context history.

It calls the **agent_query_understanding** to determine the user's intent.

It then uses a simple if/elif block to route the request:

If the intent is explain, it calls **agent_explainer**.

If the intent is research, it calls **agent_research**.

If the intent is math, it calls the non-LLM calculator tool for direct computation.

If the intent doesn't match a specialized task (like chat), it falls back to the base llm wrapper for a general conversational response.

In [7]:
async def router(user_input):
    session_state["messages"].append(user_input)
    session_state["messages"] = compact_context(session_state["messages"])

    intent = (await agent_query_understanding(user_input)).lower()

    if "explain" in intent:
        return await agent_explainer(user_input)
    elif "summarize" in intent:
        return await agent_summary(user_input)
    elif "quiz" in intent:
        return await agent_quiz(user_input, session_state["difficulty"])
    elif "research" in intent:
        return await agent_research(user_input)
    elif "math" in intent:
        result = await calculator(user_input)
        return f"Math Result: {result}"
    else:
        return await llm(user_input)

## MAIN EXECUTION LOOP
This cell contains the *synapse_scholar()* function, which is the main conversational driver. It runs an infinite while True loop that continuously prompts the user for input. When the user types **"exit"**, the loop breaks. For every query, it records the start time, calls the router to get the response, records the end time, and prints the response along with the total latency (response time). The final line, **await synapse_scholar()**, executes the application and starts the chat session.

In [8]:
async def synapse_scholar():
    print("🧠 Synapse Scholar Online. Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break

        start = time.time()
        response = await router(user_input)
        end = time.time()

        print(f"\nSynapse Scholar ({end-start:.2f}s):\n{response}\n")

await synapse_scholar()

🧠 Synapse Scholar Online. Type 'exit' to quit.



You:  who was steve jobs?


[LOG] Query Understanding Agent triggered
[LOG] Explainer Agent triggered

Synapse Scholar (2.93s):
Okay, imagine you have an iPhone, iPad, or use a Mac computer. These cool gadgets are famous for being easy to use and looking great.

Steve Jobs was the guy who was the *big boss* behind these things. He was the co-founder and CEO of a company called Apple.

Think of it like this:

*   **He had a vision:** He imagined technology that was simple, beautiful, and changed the way people lived. He didn't just want to make computers; he wanted to make *amazing* computers that everyone wanted.

*   **He was a perfectionist:** He cared a lot about the details, even the tiny ones, to make sure everything was perfect. He pushed his team really hard to make the best products possible.

*   **He was good at presenting:** He knew how to show off his products and get people excited about them. He was famous for his presentations where he would unveil new Apple devices.

So, in short, Steve Jobs was a

You:  exit


## Reporting and Metrics Display
This is a utility cell designed to be run after the chat session has ended. It prints a structured, easy-to-read summary of the session using the data collected in Cells 3 and 4:

1. It displays the metrics (tool calls, agent calls).

2. It displays the contents of the memory_bank.

It displays the last 10 detailed logs from the session, which is invaluable for post-session analysis and debugging the agent workflow.

In [9]:
print("====LOGS====")

print("📊 METRICS")
print(json.dumps(metrics, indent=2))

print("\n📝 MEMORY BANK")
print(json.dumps(memory_bank, indent=2))

print("\n🔍 LOGS (Last 10)")
print(json.dumps(logs[-10:], indent=2))

====LOGS====
📊 METRICS
{
  "total_agent_calls": 2,
  "tool_calls": 0,
  "parallel_runs": 0,
  "sequential_runs": 0
}

📝 MEMORY BANK
[]

🔍 LOGS (Last 10)
[
  {
    "event": "Query Understanding Agent triggered",
    "extra": null,
    "time": "2025-11-19T20:29:35.338882"
  },
  {
    "event": "Explainer Agent triggered",
    "extra": null,
    "time": "2025-11-19T20:29:36.135667"
  }
]
